#### Важное требование ко всей домашке в целом: в jupyter ноутбуке не должно был лишнего кода (т.е. если вы взяли за основу семинар, не забудьте удалить все лишнее)

In [ ]:
!pip install pymorphy2[fast]

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

from IPython.display import Image
from IPython.core.display import HTML 
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
wiki0 = open('/content/drive/MyDrive/hw6/wiki_data.txt').read().split('\n')

In [ ]:
wiki = open('/content/drive/MyDrive/hw6/wiki_data.txt').read().splitlines()[:5000]

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window не так используется)  
3) измените часть с np.random.randint(vocab_size) так, чтобы случайные негативные примеры выбирались обратно пропорционально частотностям слов (частотные должны выбираться реже, а редкие чаще)

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

In [ ]:
#лемматизация с помощью PyMorphy
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

In [ ]:
vocab = Counter()
for text in wiki:
    vocab.update(preprocess(text))

In [ ]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

len(filtered_vocab)

4053

In [ ]:
word2id = { 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [ ]:
id2word = {i:word for word, i in word2id.items()}

In [ ]:
sentences = []

for text in wiki:
    tokens = preprocess(text)
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

In [ ]:
vocab_size = len(id2word)

In [ ]:
# skip gram
def gen_batches_sg(sentences, window, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window//2):i] + sent[i+1:i+window//2] #делю окно пополам, т.к. его размер кратен 2, а целое окно даст нам в два раза больше контекста
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)
                    
                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)
                    
                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [ ]:
# cbow 
def gen_batches_cbow(sentences, window, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window//2):i] + sent[i+1:i+window//2] #делю окно пополам, т.к. его размер кратен 2, а целое окно даст нам в два раза больше контекста

                X_target.append(word)
                X_context.append(context)
                y.append(1)
                
                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window*2)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [ ]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(1,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)

dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(gen_batches_sg(sentences[:4000], window=12),
          validation_data=gen_batches_sg(sentences[4000:],  window=12),
          batch_size=1000,
          steps_per_epoch=10000,
          validation_steps=30,
         epochs=2)

Epoch 1/2
10000/10000 [==============================] - 324s 32ms/step - loss: 0.4768 - accuracy: 0.7755 - val_loss: 0.4973 - val_accuracy: 0.7570
Epoch 2/2
10000/10000 [==============================] - 323s 32ms/step - loss: 0.4562 - accuracy: 0.7868 - val_loss: 0.5008 - val_accuracy: 0.7654


In [ ]:
embeddings_sg = model.layers[2].get_weights()[0]

In [ ]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in 
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

In [ ]:
#cbow negative sampling
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

# полученную близость нужно преобразовать в вероятность
# когда она одна используется не софтмакс и сигмоида
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
model.fit(gen_batches_cbow(sentences[:4000], window=6),
          validation_data=gen_batches_cbow(sentences[4000:],  window=6),
          batch_size=1000,
          steps_per_epoch=5000,
          validation_steps=30,
         epochs=2)

Epoch 1/2
5000/5000 [==============================] - 230s 46ms/step - loss: 0.3532 - accuracy: 0.8418 - val_loss: 0.3550 - val_accuracy: 0.8509
Epoch 2/2
5000/5000 [==============================] - 230s 46ms/step - loss: 0.2095 - accuracy: 0.9191 - val_loss: 0.4444 - val_accuracy: 0.8385


In [ ]:
embeddings_cbow = model.layers[2].get_weights()[0]

In [ ]:
most_similar('птица', embeddings_sg)

['птица',
 'растение',
 'питаться',
 'яйцо',
 'см.',
 'произрастать',
 'тонкий',
 'личинка',
 'узкий',
 'семя']

In [ ]:
most_similar('птица', embeddings_cbow)

['птица',
 'рыба',
 'цвета,',
 'мм.',
 'произрастать',
 'старинный',
 'обитать',
 'полосатый',
 'протекать',
 'хвост']

In [ ]:
most_similar('любовь', embeddings_sg)

['любовь',
 'ты',
 '«я',
 'рассказывать',
 'мой',
 'утверждает,',
 'написание',
 'персонаж',
 'чувство',
 'текст']

In [ ]:
most_similar('любовь', embeddings_cbow)

['любовь',
 'бог',
 'валерий',
 'присоединиться',
 'обращаться',
 'приговорить',
 'бадалбейль',
 'маргарита',
 'готовиться',
 'алла']

In [ ]:
most_similar('нос', embeddings_sg)

['нос',
 'скорость',
 'равный',
 'вес',
 'меньше,',
 'максимальный',
 'превышать',
 'параллельный',
 'немного',
 'поверхность']

In [ ]:
most_similar('нос', embeddings_cbow)

['нос',
 'мм',
 'меч',
 'окраска',
 'мм,',
 'мм.',
 'москва)',
 'года.##############################',
 'женщины.##############################',
 'PAD']

In [ ]:
most_similar('наука', embeddings_sg)

['наука',
 'наук,',
 'академик',
 'физика',
 'гуманитарный',
 'специальность',
 'учёный',
 'факультет',
 'институт',
 'философия']

In [ ]:
most_similar('наука', embeddings_cbow)

['наука',
 'наук,',
 'архитектура',
 'университета.',
 'искусства,',
 'медицина',
 'академик',
 'экономика',
 'гуманитарный',
 'медицинский']

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [ ]:
import gensim

In [ ]:
def preprocess_simple(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [ ]:
texts = [preprocess_simple(text) for text in wiki]

## Word2Vec

In [ ]:
%%time
w2v = gensim.models.Word2Vec(texts, 
                             vector_size=200,
                             min_count=40,
                             max_vocab_size=15000,
                             window=6,
                             epochs=5,
                             sg=1,
                             hs=0,
                             negative=6,
                             sample=1e-4,
                             ns_exponent=0)

CPU times: user 26.2 s, sys: 89.5 ms, total: 26.3 s
Wall time: 14.4 s


In [ ]:
w2v.wv.most_similar('церковь')

[('построена', 0.9310739636421204),
 ('храм', 0.9217038154602051),
 ('храма', 0.9060737490653992),
 ('собора', 0.8876604437828064),
 ('монастыря', 0.8784587979316711),
 ('собор', 0.8770540952682495),
 ('святого', 0.8611245155334473),
 ('монастырь', 0.8600257039070129),
 ('николая', 0.8557025194168091),
 ('здание', 0.8465665578842163)]

In [ ]:
w2v.wv.most_similar('мать')

[('отец', 0.9471056461334229),
 ('семья', 0.9322947859764099),
 ('отца', 0.9087460041046143),
 ('жены', 0.8901433348655701),
 ('брат', 0.8825151324272156),
 ('брата', 0.8747135996818542),
 ('ленни', 0.8596625328063965),
 ('женат', 0.8505533933639526),
 ('матери', 0.8487252593040466),
 ('сына', 0.8481094837188721)]

In [ ]:
w2v.wv.most_similar('дом')

[('улице', 0.8870434761047363),
 ('здании', 0.8746968507766724),
 ('дома', 0.8675661683082581),
 ('сад', 0.8615222573280334),
 ('здание', 0.8384958505630493),
 ('архитектора', 0.8379758596420288),
 ('архитектор', 0.810909628868103),
 ('церковь', 0.7955955266952515),
 ('отделение', 0.7913660407066345),
 ('установлен', 0.7890158295631409)]

## FastText

In [ ]:
ft = gensim.models.FastText(texts, 
                            vector_size=300, 
                            min_count=50, 
                            max_vocab_size=17000,
                            window=7,
                            epochs=7,
                            sg=0,
                            hs=1,
                            cbow_mean=0,
                            sample=1e-4,
                            min_n=2, 
                            max_n=10)

In [ ]:
ft.wv.most_similar('церковь')

[('церкви', 0.8058878183364868),
 ('храма', 0.7765364646911621),
 ('храм', 0.7610180377960205),
 ('собора', 0.7036079168319702),
 ('николая', 0.6871659755706787),
 ('святого', 0.682386577129364),
 ('святой', 0.6788768768310547),
 ('память', 0.6535791754722595),
 ('построена', 0.6491827368736267),
 ('петра', 0.6444600820541382)]

In [ ]:
ft.wv.most_similar('мать')

[('матери', 0.8273817300796509),
 ('сына', 0.764737606048584),
 ('семь', 0.7561725378036499),
 ('играть', 0.754237949848175),
 ('семья', 0.754179835319519),
 ('анна', 0.746289074420929),
 ('стать', 0.7456331849098206),
 ('семьи', 0.7429718375205994),
 ('матча', 0.7401031255722046),
 ('мария', 0.7356366515159607)]

In [ ]:
ft.wv.most_similar('дом')

[('дома', 0.9207866191864014),
 ('доме', 0.9025293588638306),
 ('домой', 0.8580576777458191),
 ('домов', 0.8421072959899902),
 ('рядом', 0.8367487192153931),
 ('до', 0.7953340411186218),
 ('городом', 0.7845953702926636),
 ('им', 0.7719498872756958),
 ('дворе', 0.7688232660293579),
 ('дворца', 0.7677013278007507)]